<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#保存模型" data-toc-modified-id="保存模型-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>保存模型</a></span></li><li><span><a href="#加载模型" data-toc-modified-id="加载模型-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>加载模型</a></span></li><li><span><a href="#实例化可还原滑动平均值的saver" data-toc-modified-id="实例化可还原滑动平均值的saver-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>实例化可还原滑动平均值的saver</a></span></li><li><span><a href="#均方误差计算方法" data-toc-modified-id="均方误差计算方法-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>均方误差计算方法</a></span></li><li><span><a href="#在损失函数中加入正则化" data-toc-modified-id="在损失函数中加入正则化-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>在损失函数中加入正则化</a></span></li><li><span><a href="#滑动平均" data-toc-modified-id="滑动平均-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>滑动平均</a></span></li></ul></div>

## 保存模型

In [ ]:
# 保存模型

saver = tf.train.Saver()    # 实例化saver对象
with tf.Session() as sess:
    for i in xrange(STEPS):
        if i % LOOP == 0:
            saver.save(sess, 
                       os.path.join(MODEL_SAVE_PATH, MODEL_NAME),
                       global_step = global_step)

## 加载模型

In [ ]:
# 加载模型

with tf.Session() as sess:
    ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)

## 实例化可还原滑动平均值的saver

In [ ]:
# 实例化可还原滑动平均值的saver

ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
ema_restore = ema.variables_to_restore()
saver = tf.train.Saver(ema_restore)

## 均方误差计算方法

In [ ]:
# 均方误差计算方法

correct_prediciton = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## 在损失函数中加入正则化

In [ ]:
# 在损失函数中加入正则化

def get_weight(shape, regularizer):
    w = tf.Variable(tf.random_normal(shape), dtype=tf.float32)
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(regularizer)(w))
    return w

ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
cem = tf.reduce_mean(ce)
loss = cem + tf.add_n(tf.get_collection("losses"))

## 滑动平均

In [ ]:
# 滑动平均

train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss_total)
ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step=global_step)
ema_op = ema.apply(tf.trainable_variables())
with tf.control_dependencies([train_step, ema_op]):
    train_op = tf.no_op(name='train')